In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import timeit


In [93]:
# data = np.genfromtxt('data.txt', delimiter=';', names=True,
#                      dtype=['S10','S8','f8','f8','f8','f8','f8','f8','f8' ]
#                     )
# data.dtype

In [15]:
#Read txt file and writing to DataFrame object

df = pd.read_table('data.txt',sep=';', header=0)

#Delete NaN values
df.replace(to_replace="?", value="NaN", inplace=True)
df = df.dropna()

#Convert to number
df[['Global_active_power', 'Global_reactive_power',
       'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3']] = df[['Global_active_power', 'Global_reactive_power',
       'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3']].apply(pd.to_numeric)

In [11]:
#Create ndarrays object to use DataFrame
arr = df.as_matrix()

In [16]:
print(len(df))
print(len(arr))

2049280
2049280


In [17]:
df.columns

Index([u'Date', u'Time', u'Global_active_power', u'Global_reactive_power',
       u'Voltage', u'Global_intensity', u'Sub_metering_1', u'Sub_metering_2',
       u'Sub_metering_3'],
      dtype='object')

    1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.

In [47]:
# timeit.Timer(
#     stmt='df.index'
#     #setup='from __main__ import df'
#             ).timeit()

print("DataFrame:")
%timeit df[df['Global_reactive_power'] > 5]

print("NumPy Array:")
%timeit arr[arr[:,2]>5]

DataFrame:
100 loops, best of 3: 2.51 ms per loop
NumPy Array:
10 loops, best of 3: 75.8 ms per loop


    2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.

In [46]:
print("DataFrame:")
%timeit df[df['Voltage'] > 235]

print("NumPy Array:")
%timeit arr[arr[:,4] > 235]

DataFrame:
10 loops, best of 3: 134 ms per loop
NumPy Array:
1 loop, best of 3: 414 ms per loop


    3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.

In [45]:
def dataFrame():
    intensity = df[(df['Global_intensity'] >= 19) & (df['Global_intensity'] <= 20)]
    return intensity[intensity['Sub_metering_2'] > intensity['Sub_metering_3']]

def arrays():
    intensity = arr[(arr[:,5] >= 19) & (arr[:,5] <= 20)]
    return intensity[intensity[:,7] > intensity[:,8]]

print("DataFrame:")
%timeit dataFrame()

print("NumPy Array:")
%timeit arrays()

DataFrame:
100 loops, best of 3: 9.67 ms per loop
NumPy Array:
10 loops, best of 3: 146 ms per loop


    4.Обрати випадковим чином 500000 домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії, а також 

In [44]:
def get_DF():
    newDF = df.ix[np.random.choice(range(len(df)), size=500000, replace=False)]
    return [np.mean(newDF['Sub_metering_1']), np.mean(newDF['Sub_metering_2']), np.mean(newDF['Sub_metering_3'])]
def get_NP():
    newNP = arr[np.random.choice(range(len(df)), size=500000, replace=False),:]
    return [np.mean(newNP[:,6]), np.mean(newNP[:,7]), np.mean(newNP[:,8])]
   
print("DataFrame:")
%timeit get_DF()

print("NumPy Array:")
%timeit get_NP()

DataFrame:
1 loop, best of 3: 336 ms per loop
NumPy Array:
1 loop, best of 3: 981 ms per loop


    5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [43]:
def five_DF():
    firstCh = df[(df['Time']>'18:00:00') & (df['Global_active_power']>6) 
                 & (df['Sub_metering_2'] > df['Sub_metering_3']) 
                 & (df['Sub_metering_2'] > df['Sub_metering_3'])
                ]   
    third = firstCh.iloc[0:len(firstCh)/2:3, :]
    fourth = firstCh.iloc[len(firstCh)/2+1::4,:]
    return third,fourth
def five_NP():
    firstNP = arr[(arr[:,1]>'18:00:00') & (arr[:,2]>6 )
                 & (arr[:,7]>arr[:,8])
                 & (arr[:,7]>arr[:,6])
                 ]
    third = firstNP[0:len(firstNP)/2:3, :]
    fourth = firstNP[len(firstNP)/2+1::4,:]
    return third, fourth
print("DataFrame:")
%timeit five_DF()

print("NumPy Array:")
%timeit five_NP()

DataFrame:
10 loops, best of 3: 87.5 ms per loop
NumPy Array:
1 loop, best of 3: 213 ms per loop


# THE END